# RouteOptimize Solutions

The airline industry struggles with optimizing flight routes and delivering personalized travel experiences.

GenAI offers advanced algorithms, real-time data analysis, and natural language processing capabilities.

RouteOptimize Solutions leverages GenAI to provide a chatbot interface for understanding natural language queries and offering customized flight recommendations.


This notebook provides a step-by-step guide for our new chatbot which we have created which leverages LLMs and provide an optimized route option to user based on his needs.

We will go through the following steps:

1. **Setup:** Loading our dataset and filtering down to one domain to fine-tune on.
2. **Data preparation:** Preparing the data for fine-tuning by creating training and validation examples.
3. **Fine-tuning:** Creating a fine-tuned model to tackle the problems.
4. **Inference:** Use the fine-tuned model for inference on new inputs.

By the end of this you should be able to provide in various prompts and receive the fastest route to carry out delivery with the shortest optimized route.


## Setup

This is the first step of our process. Here we will import various libraries and packages so that further functions and code can leverage these libraries and packages to run the code effectively

In [ ]:
# Installing the OpenAI Python package
!pip install --upgrade openai

In [ ]:
#Mounting the google drive for use
from google.colab import files, drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Imorting variouos other libraries
import json
import openai
import os
import pandas as pd
from pprint import pprint

# Initialize the OpenAI client object with the api_key parameter is used for authentication with OpenAI's API.
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-BjbN5kOrr11q6qacmeJZT3BlbkFJcGG2jjJfZ7wgg8RLVnTV"))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data preparation

We will prepare our dataset by cleaning and making sure it is ready to be used to create a model. We will carry out various kinds of data cleaning and pre processing like dealing with null values, outliers and much more so that there are no discrepancies in the model we create using that data.

In [ ]:
#Reading our dataset of various airline routes which would be used further
airline_df = pd.read_csv("/content/drive/MyDrive/AI Project/final_dataset.csv")

airline_df.head()

,Start Date,Departure Airport,Departure City,Departure Country,Destination Airport,Destination City,Destination Country,Layover Airport,Layover City,Layover Country,Layover Time,Flight Duration,Airline,Flight Number,NER
0,10/08/2023,Ie Shima Auxiliary Air Base,Iejima,Japan,Sandefjord Airport,Torp,Sandefjord,Roudnice Airport,Roudnice nad Lebem,Czech Republic,4.0,6.0,Alexander-Byrd,EAU460,Ie Shima Auxiliary Air Base\nIejima\nJapan\nSa...
1,08/20/2023,Mayor Buenaventura Vivas International Airport,Santo Domingo,Venezuela,Akhiok Airport,Akhiok,United States,CapitÃ¡n Manuel RÃ­os Airbase,Carrizal,Venezuela,5.0,1.0,Cole and Sons,EIZ862,Mayor Buenaventura Vivas International Airport...
2,11/04/2023,Berbera Airport,Berbera,Somalia,Majors Airport,Greenvile,United States,Roxas Airport,Roxas City,Philippines,4.0,12.0,Morgan Inc,RXD676,Berbera Airport\nBerbera\nSomalia\nMajors Airp...
3,09/01/2023,Roshchino International Airport,Tyumen,Russia,Aalborg Airport,Aalborg,Denmark,Benin Airport,Benin,Nigeria,6.0,11.0,Lucero-Lewis,HGE223,Roshchino International Airport\nTyumen\nRussi...
4,05/27/2023,Brustem Airfield Sint Truiden,Sint-truiden,Belgium,Kenneth Kaunda International Airport Lusaka,Lusaka,Zambia,Cozumel International Airport,Cozumel,Mexico,4.0,1.0,Bernard LLC,JRO437,Brustem Airfield Sint Truiden\nSint-truiden\nB...


In [ ]:
# Initialize an empty list to store training data
training_data = []

# We define a system message providing instructions to the assistant which informs the assistant about its role and what it's expected to do.
system_message = "You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total travel duration."

# Define a function to create a user message based on a given row of data.
# The message includes Start Date, Departure City, and Destination City extracted from the row.
def create_user_message(row):
    message = f"Start Date: {row['Start Date']}\n"
    message += f"Departure City: {row['Departure City']}\n"
    message += f"Destination City: {row['Destination City']}\n\n"
    message += "Based on the given information, please provide the most suitable travel plan."
    return message

# Next we define a function to prepare an example conversation for training.
# It constructs a conversation with a system message, user message, and assistant response based on the given row of data.
def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})
    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": row["NER"]})
    return {"messages": messages}

#Now we iterate over each row in the airline dataset and create training examples.
# For each row, prepare an example conversation and append it to the training data list.
for index, row in airline_df.iterrows():
    training_data.append(prepare_example_conversation(row))

Now do this for a subset of the dataset to use as our training data. We see performance continue to scale linearly as we increase the size of the training set, but the jobs also take longer.

In [ ]:
# use the first 100 rows of the dataset for training
training_df = airline_df.loc[0:100]

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total travel duration.'}, {'role': 'user', 'content': 'Start Date: 10/08/2023\nDeparture City: Iejima\nDestination City: Torp\n\nBased on the given information, please provide the most suitable travel plan.'}, {'role': 'assistant', 'content': 'Ie Shima Auxiliary Air Base\nIejima\nJapan\nSandefjord Airport\nTorp\nSandefjord\nAlexander-Byrd\nEAU460\nRoudnice Airport\nRoudnice nad Lebem\nCzech Republic\n4.0\n6.0'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total tra

We carry out validation of our data which will make sure that the model does not overfit the training set.

In [ ]:
validation_df = airline_df.loc[101:200]
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

We define a function to write a list of dictionaries to a JSON Lines file.
JSON Lines (JSONL) is a format where each line is a separate JSON object.
This function takes two parameters: data_list, a list of dictionaries to be written to the file, and filename, the name of the file where the data will be saved.

In [ ]:
#defining a function write_json
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
          #We convert the dictionary to a JSON string and add a newline character.
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

Now we call the function which we created to convert all files to JSON Lines

In [ ]:
training_file_name = "airline_details_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "airline_details_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [ ]:
#This is what after conversion our file looks like
!head -n 5 airline_details_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total travel duration."}, {"role": "user", "content": "Start Date: 10/08/2023\nDeparture City: Iejima\nDestination City: Torp\n\nBased on the given information, please provide the most suitable travel plan."}, {"role": "assistant", "content": "Ie Shima Auxiliary Air Base\nIejima\nJapan\nSandefjord Airport\nTorp\nSandefjord\nAlexander-Byrd\nEAU460\nRoudnice Airport\nRoudnice nad Lebem\nCzech Republic\n4.0\n6.0"}]}
{"messages": [{"role": "system", "content": "You are a helpful travel assistant. Given the Start Date, Departure City, Destination City, you will provide the shortest travel details, including departure airport, destination airport, airline, flight number, layover details (if any), and total tra

This code segment opens the training and validation data files in binary read mode and utilizes the OpenAI client to create file objects for each dataset, indicating their purpose as "fine-tuning" the model. By extracting the IDs from the responses, it establishes unique identifiers for both the training and validation files. Finally, it prints these IDs for reference, marking the completion of the file upload process to OpenAI's service.

In [ ]:
# Open the training data file in binary read mode.
with open(training_file_name, "rb") as training_fd:

#We use the OpenAI client to create a file object for the training data using the data read from the training data file.
# The purpose parameter is set to "fine-tune" to indicate that the data will be used for fine-tuning the model.
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

# Extracting the ID of the training file from the response.
training_file_id = training_response.id

# Open the validation data file in binary read mode.
with open(validation_file_name, "rb") as validation_fd:

#We use the OpenAI client to create a file object for the training data using the data read from the validation data file.
# The purpose parameter is set to "fine-tune" to indicate that the data will be used for fine-tuning the model.
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )

# Extracting the ID of the training file from the response.
validation_file_id = validation_response.id

#Printing thr training and validation file id
print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-8Al9M6zO6aN4bkLNhpKhPfAa
Validation file ID: file-iP76Kf9P32ONYx2DiYzIlQau


## Fine-tuning



Now we start the process of fine tuning our model by creating a job using OpenAI which uses all the data and tries to make our model better.

In [ ]:
# Initiate the fine-tuning process by creating a job using the OpenAI client.
# Parameters include the IDs of the training and validation files, the model to be fine-tuned ("gpt-3.5-turbo"),
# and a suffix to identify the specific fine-tuning task ("airline-ner" in this case).
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix="airline-ner",
)

# Extract the ID of the fine-tuning job from the response.
job_id = response.id

# Print the ID and status of the initiated job for reference.
print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-sEYVdJHf5vspkMdlL2mfGDlf
Status: validating_files


#### Check job status



In [ ]:
# Retrieve information about the fine-tuning job using the OpenAI client.
# The job information is retrieved based on the provided job ID.
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-sEYVdJHf5vspkMdlL2mfGDlf
Status: validating_files
Trained Tokens: None


In [ ]:
# Retrieve a list of events associated with the fine-tuning job using the OpenAI client.
# The events provide detailed information about the progress and actions taken during the fine-tuning process.
response = client.fine_tuning.jobs.list_events(job_id)

# Extract the event data from the response.
events = response.data

#Reverse the order of events to display them chronologically from the latest to the earliest.
events.reverse()

for event in events:
    print(event.message)

Step 288/303: training loss=0.84
Step 289/303: training loss=0.86
Step 290/303: training loss=0.60, validation loss=0.78
Step 291/303: training loss=0.87
Step 292/303: training loss=0.72
Step 293/303: training loss=0.79
Step 294/303: training loss=0.78
Step 295/303: training loss=0.83
Step 296/303: training loss=0.87
Step 297/303: training loss=0.84
Step 298/303: training loss=0.90
Step 299/303: training loss=0.65
Step 300/303: training loss=0.70, validation loss=0.68
Step 301/303: training loss=0.83
Step 302/303: training loss=0.71
Step 303/303: training loss=0.62, full validation loss=0.85
Checkpoint created at step 101 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:airline-ner:9O4dPUzv:ckpt-step-101
Checkpoint created at step 202 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:airline-ner:9O4dQ2GW:ckpt-step-202
New fine-tuned model created: ft:gpt-3.5-turbo-0125:personal:airline-ner:9O4dQ83j
The job has successfully completed


Now we can get a fine-tuned model ID from the job:


In [ ]:
# Retrieve information about the fine-tuning job using the OpenAI client.
# The job information is retrieved based on the provided job ID.
response = client.fine_tuning.jobs.retrieve(job_id)

# Extract the ID of the fine-tuned model from the response.
fine_tuned_model_id = response.fine_tuned_model

# If the ID is None, it indicates that the fine-tuning job has not been completed yet.
# In such a case, raise a RuntimeError to indicate that the fine-tuned model ID is not found.
if fine_tuned_model_id is None:
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:airline-ner:9O4dQ83j


The last step is to use the fine-tuned model for inference.


In [ ]:
# Extract a subset of rows from the airline dataset for testing purposes.
# Here, rows 201 to 300 are selected for testing.
test_df = airline_df.loc[201:300]
test_row = test_df.iloc[0]

# Initialize an empty list to store test messages, which simulate a conversation for testing purposes.
test_messages = []

# Add a system message to the test messages list which provides instructions or information to the chatbot about its role or task.
test_messages.append({"role": "system", "content": system_message})

# Create a user message based on the selected row of data using the create_user_message function.
# This user message includes details like Start Date, Departure City, and Destination City.
user_message = create_user_message(test_row)

# Add the user message to the test messages list, marking the user's input in the simulated conversation.
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are a helpful travel assistant. Given the Start Date, '
             'Departure City, Destination City, you will provide the shortest '
             'travel details, including departure airport, destination '
             'airport, airline, flight number, layover details (if any), and '
             'total travel duration.',
  'role': 'system'},
 {'content': 'Start Date: 06/21/2023\n'
             'Departure City: Everett\n'
             'Destination City: Nova Iguacu\n'
             '\n'
             'Based on the given information, please provide the most suitable '
             'travel plan.',
  'role': 'user'}]


In [ ]:
# Generate a response from the fine-tuned chatbot model using the OpenAI client.
# The response is generated based on the provided fine-tuned model ID, simulated test messages, and additional parameters.
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

Snohomish County (Paine Field) Airport
Everett
United States
Aeroporto de Nova Iguacu
Nova Iguacu
Brazil
Hernandez, Johnson and Johnson
QZP228
Kampong Bunsar Airport
Kampong Bunsar
Malaysia
4.0
6.0


We finally set OpenAI parameters for model completion and retrieve completions from the OpenAI API. It sets up a test scenario using a fine-tuned model for a travel assistance task. The provided prompt and messages simulate a conversation between the user and the travel assistant chatbot. Finally, it generates a completion using the fine-tuned model and prints both the prompt and the generated completion output for evaluation.

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "sk-proj-BjbN5kOrr11q6qacmeJZT3BlbkFJcGG2jjJfZ7wgg8RLVnTV"

# Define a function to set OpenAI parameters for model completion.
def set_open_params(model=None, temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0):
    """Set OpenAI parameters"""
    openai_params = {}

# Check if a valid model ID is provided.
    if model is None:
        print("Please provide a valid fine-tuned model ID.")
        return None

# Set model-specific parameters.
    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty

# Return the completion message from the API response.
    return openai_params

# Define a function to retrieve completion from the OpenAI API.
def get_completion(params, messages):
    """GET completion from OpenAI API"""

# Generate a completion using the provided parameters and messages.
    response = openai.chat.completions.create(
        model=params['model'],
        messages=messages,
        temperature=params['temperature'],
        max_tokens=params['max_tokens'],
        top_p=params['top_p'],
        frequency_penalty=params['frequency_penalty'],
        presence_penalty=params['presence_penalty']
    )
# Return the completion message from the API response.
    return response.choices[0].message



# Set OpenAI parameters for model completion using the defined function.
params = set_open_params(model=fine_tuned_model_id)

# If valid parameters are set, simulate a conversation and generate a completion using the model.
if params is not None:
# Define a prompt message for the conversation.
    prompt = "07/15/2023,Guarapuava,Abu Dhabi\n\nBased on the given information, please provide the shortest travel details."

# Define messages for the conversation including a system message and the user prompt.
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]

# Generate a completion based on the provided parameters and messages.
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 07/15/2023,Guarapuava,Abu Dhabi

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Tancredo Thomas de Faria Airport, Guarapuava, Brazil\nGuarapuava\nBrazil\nAl Bateen Executive Airport, Abu Dhabi, United Arab Emirates\nAbu Dhabi\nUnited Arab Emirates\n7928.0\n7.0\n-25.45\n-49.65\n88.0\n3.0', role='assistant', function_call=None, tool_calls=None)


Testing out various prompts to check if we get accurate responses. Carrying out some alpha testing to make sure our model is giving useful correct real time output

In [ ]:
if params is not None:
    prompt = "10/15/2023,Beverly,Dirkou\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 10/15/2023,Beverly,Dirkou

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Birni N Konni Airport, Birni N Konni, Niger', role='assistant', function_call=None, tool_calls=None)


In [ ]:
if params is not None:
    prompt = "08/20/2023,Quilpie,Tepic\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 08/20/2023,Quilpie,Tepic

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Quilpie Airport, Quilpie, Australia', role='assistant', function_call=None, tool_calls=None)


In [ ]:
if params is not None:
    prompt = "05/30/2023,Liege,Mercury\n\nBased on the given information, please provide the shortest travel details."
    messages = [{"role": "system", "content": "You are a helpful travel assistant."}, {"role": "user", "content": prompt}]
    output = get_completion(params, messages)
    print("Prompt:", prompt)
    print("Output:", output)

Prompt: 05/30/2023,Liege,Mercury

Based on the given information, please provide the shortest travel details.
Output: ChatCompletionMessage(content='Liege, Mercury', role='assistant', function_call=None, tool_calls=None)
